In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import glob

import sklearn
from sklearn.metrics import confusion_matrix

In [ ]:
data = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")

In [ ]:
data['label'] = data['label'].astype('string')
data['iamge_id'] = data['image_id'].astype('string')

In [ ]:
def zoom_images(tensor):
    tensor = tf.image.central_crop(tensor, 0.80)
    return tensor

In [ ]:
# for image in os.listdir('../input/cassava-leaf-disease-classification/train_images')[4:5]:
#     i = plt.imread(os.path.join('../input/cassava-leaf-disease-classification/train_images', image)) / 255.0
#     i = tf.image.resize(i, (400, 400))
#     print(zoom_images(i).shape)
#     plt.imshow(zoom_images(i))

In [ ]:
# random_indexes = np.random.randint(0, 2000, 8)
# __dir = os.listdir('../input/cassava-leaf-disease-classification/train_images')
# li = [__dir[i] for i in random_indexes]
# fig, axes = plt.subplots(8, 2, figsize=(50, 50))
# count = 0
# for k in range(8):
#     img = plt.imread('../input/cassava-leaf-disease-classification/train_images/' + li[count]) / 255.0
#     for j in range(2):
#         axes[k][j].imshow(img)
#         img = zoom_images(img)
#         axes[k][j].imshow(img)
#     count += 1

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255., horizontal_flip=True, vertical_flip=True, fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(data[:int(len(data)*0.8)], x_col='image_id', y_col='label',
                                                    directory='../input/cassava-leaf-disease-classification/train_images', 
                                                    batch_size=256, class_mode='categorical', validate_filenames=False, 
                                                    target_size=(400, 400), preprocessing_function=zoom_images)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255.)

validation_generator = validation_datagen.flow_from_dataframe(data[int(len(data)*0.8):], x_col='image_id', y_col='label', 
                                                              directory='../input/cassava-leaf-disease-classification/train_images', 
                                                              batch_size=256, class_mode='categorical', validate_filenames=False, 
                                                              target_size=(400, 400), preprocessing_function=zoom_images)

In [ ]:
model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

In [ ]:
def build_model(pre_trained_model):
    
    for layer in pre_trained_model.layers:
        layer.trainable=False
    
    
    
    x = pre_trained_model.outputs[0]
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(5, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=[pre_trained_model.inputs], outputs=[outputs], name='test_model')
    return model

model = build_model(model)

In [ ]:
# model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='recall', patience=2, mode='max', restore_best_weights=True, verbose=1
)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=METRICS)

In [ ]:
history = model.fit(train_generator, epochs=10, callbacks=[early_stopping], validation_data=validation_generator)

In [ ]:
# epochs = np.arange(len(history.history['loss']))
# fig, axes = plt.subplots(1, 2, figsize=(15, 10))
# axes[0].plot(epochs, history.history['loss'], color='r', label='training_loss')
# axes[0].plot(epochs, history.history['val_loss'], color='b', label='validation_loss')
# axes[0].legend()

# axes[1].plot(epochs, history.history['accuracy'], color='r', label='training_acuuracy')
# axes[1].plot(epochs, history.history['val_accuracy'], color='b', label='validation_accuracy')
# axes[1].legend()

# plt.show()

In [ ]:
data_test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
def final_predict(test_img, model):
    test_image = plt.imread(test_img) / 255.0
    test_image = tf.image.resize(test_image, (400, 400))
    test_image = zoom_images(test_image)
    test_image = tf.expand_dims(test_image, 0)
    prediction = np.argmax(model.predict(test_image))
    pred_df = pd.DataFrame({"image_id": ['2216849948.jpg'], "label": [prediction]})
    
    return pred_df

prediction_df = final_predict('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg', model)


In [ ]:
prediction_df.to_csv('Final_Submission.csv')